In [ ]:
# Import some packages that we need
import matplotlib.pyplot as plt
import numpy as np

import flopy as fp

plt.rcParams["figure.autolayout"] = True # Produces better figures

# Groundwater Modelling in Python

## Analytical solutions

The Theis solution is a famous formula in groundwater hydrology to compute the drawdown around a well in a confined aquifer. The aquifer is approximated as fully-confined and the transmissivity $T$ and storage coefficient $S$ are approximated as constant. The well extraction rate is $Q$.

The solution for a well that starts pumping at $t=0$ is
\begin{equation}
h = - \frac{Q}{4\pi T}\text{E}_1\left( u\right) {\hskip 2em} t\ge t_0
\end{equation}
where
\begin{equation}
u =  \frac{S r^2}{4Tt}
\end{equation}
and $\text{E}_1$ is the exponential integral
\begin{equation}
\text{E}_1(u) = \int_u^{\infty} \frac{\exp(-s)}{s}\text{d} s 
\end{equation}

Because the pumped water can only come from storage in the fully confined aquifer, the Theis solution does not reach steady state. It is only suitable to analyze the early stages of a pumping test, before other sources of water (i.e. from an overlying aquifer) start to contribute significantly to the well flow. 

The following code cell shows how the Theis solution is implemented in Python

In [ ]:
from scipy.special import exp1 # Import the exponential integral from SciPy

# Implement the formula as a function
def theis(r, t, T, S, Q):
    u = S * r ** 2 / (4 * T * t)
    return -Q / (4 * np.pi * T) * exp1(u)

Next, set the numerical values for the parameters

In [ ]:
# parameters
k = 10  # hydraulic conductivity, m/d
H = 10  # aquifer thickness, m
T = k * H  # transmissivity of aquifer, m^2/d
S = 0.001  # storage coefficient of aquifer, -
Q = 800  # discharge of well, m^3/d
rw = 0.3  # radius of well, m

We'll plot two graphs: One with the head as a function of distance from the well after $t = 0.5$ day, and one that shows the head as a function of time at a distance of $r = 40$ m.

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))

ax = axs[0]
r = np.linspace(rw, 1000, 100) # radial distance from the well, m
t = 0.5 # time, d
h_theis = theis(r, t, T, S, Q)
ax.plot(r, h_theis, label="Theis, 0.5 d")

ax.grid()
ax.legend()
ax.set_xlabel("$r$ (m)")
ax.set_ylabel("head (m)")
ax.set_xlim(0, 500)
ax.set_ylim(-5, 0)

ax = axs[1]

r = 40 # m
t = np.logspace(-3, np.log10(12 / 24), 100)
h_theis = theis(r, t, T, S, Q)
ax.semilogx(t, h_theis, label="Theis, $r=40$ m")

ax.grid()
ax.legend()
ax.set_xlabel("time (d)")
ax.set_ylabel("head (m)")
ax.set_ylim(-3, 0);

Let's now try to model the Theis solution in MODFLOW 6. The first step in creating a MODFLOW 6 model is to create the simulation. 

In [ ]:
modelname = "model_theis"  # model name to be used
modelws = "./model_theis"  # model work space to be used

sim = fp.mf6.MFSimulation(
    sim_name=modelname,  # name of simulation
    sim_ws=modelws,  # path to workspace where all files are stored
    exe_name="../bin/mf6.exe",  # path to MODFLOW executable
)

The next step is to discretize the time. For an accurate solution it is important to accurately resolve the earliest times after the pumping starts. This is achieved by using 20 irregular time steps by setting the time step multiplier to 1.2.

In [ ]:
perlen = 0.5  # stress period length, d
nstp = 20  # number of time steps
tsmult = 1.2  # time step multiplier

tdis = fp.mf6.ModflowTdis(
    simulation=sim,
    time_units="DAYS",
    nper=1,  # number of stress periods
    perioddata=[(perlen, nstp, tsmult)],
)

The function `ModflowIms` defines the solver settings. There are many parameters that can be tweaked but for this demo, we'll just use the defaults.

In [ ]:
ims = fp.mf6.ModflowIms(
    sim,
)

Creating the groundwater flow model is done by calling the `ModflowGwf` function. The simulation to which the model belongs must always be specified.

In [ ]:
gwf = fp.mf6.ModflowGwf(
    simulation=sim,
    modelname=modelname,  # model name
)

A challenge in setting up the numerical model is to determine the extent of the model domain. The Theis function is for a semi-infinite aquifer, but in MODFLOW 6 we obviously have a finite model extent. By playing around with the analytical solution, it can be determined that it is sufficient to extend the model 1000 m away from the well. We choose a model extent of 2040 m and start out with a cell size of 40 m, with the well placed in the central model cell.

In [ ]:
Lx = 2040  # West-East length of domain, m
Ly = 2040  # North-South length of domain, m

# Well location
xwell = Lx / 2
ywell = Ly / 2

A grid of 51 rows and 51 columns is used, resulting in the desired cell size of 40 m in both the row and column directions. The top of the aquifer is at 0 meters, so the bottom is at -10 m.

In [ ]:
# Layers
nlay = 1  # number of layers
top = 0  # top of top aquifer layer, m
botm = top - H  # aquifer bottom, m

# Grid properties
ncol = 51  # number of columns
nrow = 51  # number of rows
delc = Ly / nrow  # cell size in column direction
delr = Lx / ncol  # cell size in row direction

dis = fp.mf6.ModflowGwfdis(
    model=gwf,
    length_units="METERS",
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)

Aquifer properties are defined with the `ModflowGwfnpf` function.

In [ ]:
npf = fp.mf6.ModflowGwfnpf(
    model=gwf,
    k=k,  # horizontal k value
)

The storage package is invoked to simulate transient flow. Note how the storage coefficient is divided by the aquifer thickness because it is the specific storage coefficient that must be passed to `ModflowGwfsto`.

In [ ]:
sto = fp.mf6.ModflowGwfsto(
    model=gwf,
    ss=S / H, # Specific storage coefficient, 1/m
    transient=True,
)

The function `ModflowGwfic` sets the intial conditions. Initially all heads are set equal to the top elevation (i.e., $h = 0$).

In [ ]:
ic = fp.mf6.ModflowGwfic(
    model=gwf,
    strt=top,  # initial head,
)

The row and column numbers of the center cell (where the well is located) are found by dividing `nrow` and `ncol` by two. The two forward slashes indicate an integer division in Python, which ensures that the result of the division is also an integer number.

In [ ]:
rwell = nrow // 2  # results in an integer
cwell = ncol // 2
spd_list = [[(0, rwell, cwell), -Q]] # a nested list which contains one list per well with well location and pumping rate
well_spd = {0: spd_list}  # dictionary of stress period data

wel = fp.mf6.ModflowGwfwel(
    model=gwf, 
    stress_period_data=well_spd,
)

Output control is specified using `ModflowGwfoc`.

In [ ]:
oc = fp.mf6.ModflowGwfoc(
    model=gwf,  # add output control to model gwf (created in previous code cell)
    budget_filerecord=f"{modelname}.cbc",  # file name where all budget output is stored
    head_filerecord=f"{modelname}.hds",  # file name where all head output is stored
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

Before running the model it is worthwhile to inspect the grid and make sure that the well is indeed in the correct location.

In [ ]:
pmv = fp.plot.PlotMapView(model=gwf)  # create map view
pmv.plot_bc(name="wel")  # show well location
pmv.plot_grid(colors="silver", lw=1)

We write the input files and run the model with the `silent` option set to `True` to reduce the amount of screen output.

In [ ]:
sim.write_simulation(silent=False)
success, dummy = sim.run_simulation(silent=True)
if success == True:
    print("Model solved successfully")
else:
    print("Solve failed")

Once the results are calculated we can read the binary head file

In [ ]:
hds = gwf.output.head()  # get handle to binary head file
head = hds.get_data()
times = hds.get_times()

In order the compare the numerical results with the analytic solution, the heads from the MODFLOW model are plotted as a function of $r$ at the end of the model simulation ($t = 0.5$ d). The lines are drawn as step plots to emphasize that this is the result from the numerical (discretized) model. The analytic solution is also included.

In the right graph, the head is plotted versus time for the cell next to the extraction well (the head in the cell containing the well is not representative for the drawdown in the well, because of the much larger size of the model cell compared to the radius of the well).

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))

ax = axs[0] # Selects the left graph

# Plot the numerical solution
r = np.arange(0, Lx / 2, delr)
t = 0.5 # time, d
ax.step(
    r,
    head[0, rwell, cwell:],
    where="mid",
    color="C0",
    label="t = 0.5 d",
)

# Plot the analytical solution
h_theis = theis(r, t, T, S, Q)
ax.plot(r, h_theis, color="C0")

# Prettify the graph
ax.grid()
ax.legend()
ax.set_xlabel("$r$ (m)")
ax.set_ylabel("head (m)")
ax.set_xlim(0, Lx / 4)
ax.set_ylim(-5, 0)

ax = axs[1] # Selects the right graph

# Plot the numerical solution
ht = hds.get_ts([[0, rwell + 1, cwell]]) # Get a time series of the heads
ax.semilogx(ht[:, 0], ht[:, 1], "C2.", label="MODFLOW 6")

# Plot the analytical solution
h_theis = theis(delr, np.array(times), T, S, Q)
ax.semilogx(times, h_theis, "C2", label="Theis, $r=40$ m")

# Prettify the graph
ax.grid()
ax.legend()
ax.set_xlabel("time (d)")
ax.set_ylabel("head (m)")
ax.set_ylim(-3, 0);

Another way to visualize the result is to  plot the head contours at the end of the simulation

In [ ]:
pmv = fp.plot.PlotMapView(model=gwf, layer=0)
pmv.plot_grid(colors="silver", lw=1)
cs = pmv.contour_array(  # contour plot of heads
    head,
    levels=np.arange(-5, 0.5, 0.2),
    linewidths=1,
    colors="k",
    linestyles="-",
)
plt.clabel(cs, fmt="%1.1f")  # add contour labels with one decimal place
pmv.plot_bc(name="wel")  # show well location

plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Head after 0.5 d")
plt.xlim(770, 1270)
plt.ylim(770, 1270)